## Real experiments for head movements

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [19]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score,f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC,LinearSVC
from sklearn.dummy import DummyClassifier
from seqlearn.perceptron import StructuredPerceptron
import sklearn_crfsuite
from sklearn_crfsuite import metrics as skcrfmetrics
from scipy.stats import pearsonr,spearmanr

In [20]:
import glob

In [21]:
windowsize="9-11-13"
memory="middle"

PATH = "C:\\Users\\zgk261\\nomco\\"+windowsize+"\\"+memory+"\\"
PATH

'C:\\Users\\zgk261\\nomco\\9-11-13\\middle\\'

In [22]:
!dir /B C:\Users\zgk261\nomco\9-11-13\middle\

F2_M4-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F2_M4-all-final2_primary_M4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F3.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F3_F6-all-final2_primary_F6.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F1.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F4_F1-all-final2_primary_F4.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBackward-HF-HeadForward--rdXY.svm.9-11-13-middle.tab
F5_F2-all-final2_primary_F2.T-Tilt-ST-SideTurn-HO-HeadOther-S-Shake-W-Waggle-N-Nod-J-Jerk-HB-HeadBac

### SKELETON TO READ FILES, SPEAKERS and PRIMARY SPEAKERS


In [23]:
speaker_set = set()

for each_file in glob.glob(PATH+"*.tab"):
    filename=each_file.split("\\")[-1]
    fileinfo=filename.split("-")
    
    primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
    speakers = fileinfo[0]
    l_speaker,r_speaker = speakers.split("_")

    speaker_set.add(primary_speaker)

In [24]:
speaker_set

{'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6'}

### SKELETON TO READ FILES AND COSS_VALIDATE

In the following way: In each iteration, we will leave one speaker out (two files), train a model and test on those two files.

In [25]:

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()


Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Speaker on focus: F3
Training data size: (181651, 28)
Testing data size: (16409, 28)

Speaker on focus: M1
Training data size: (180025, 28)
Testing data size: (18035, 28)

Speaker on focus: M5
Training data size: (180649, 28)
Testing data size: (17411, 28)

Speaker on focus: F4
Training data size: (181137, 28)
Testing data size: (16923, 28)

Speaker on focus: M3
Training data size: (181426, 28)
Testing data size: (16634, 28)

Speaker on focus: F1
Training data size: (181919, 28)
Testing data size: (16141, 28)

Speaker on focus: F6
Training data size: (182822, 28)
Testing data size: (15238, 28)

Speaker on focus: M2
Training data size: (182715, 28)
Testing data size: (15345, 28)

Speaker on focus: F2
Training data size: (181593, 28)


### Experiments:

It seems that in the 3-6-9 experiments considering both the past and future of frames, the MLP was not the best model, but the Logistic Regression. I think, though, that the MLP will show good performance when cross-validating with the whole data.

I will, then, try 3 classifiers:

  * SVM, because it is the default classifier that is commonly used
  * Logistic Regression, because it seems to perform the best in our initial experiments with two dialogue data
  * MLP, because when including more data, I expect to estimate a more general (and thus, better) model

In [26]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
import math

def map_movements(mov_class):
    if mov_class == "Nod" or mov_class == "Jerk":
        return 1
    elif mov_class == "Shake":
        return 2
    elif mov_class!=mov_class: #This returns True if the value is NAN, and it works. It might be source of errors.
        return 0
    else:
        return 3



In [28]:
accuracy_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_results = pd.DataFrame(0,columns=speaker_set,index=["MF"])
f1score_resultsw = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1score_resultsb = pd.DataFrame(0,columns=speaker_set,index=["MF"])
#f1_score(y_true, y_pred, average='micro')

for speaker_on_focus in speaker_set:
    train_files = []
    test_files  = []
    
    for each_file in glob.glob(PATH+"*.tab"):
        filename=each_file.split("\\")[-1]
        fileinfo=filename.split("-")

        primary_speaker = fileinfo[2].split("_")[-1][:-2] #fileinfo[2] = final2_primary_M1.T
        speakers = fileinfo[0]
        l_speaker,r_speaker = speakers.split("_")
        if primary_speaker == l_speaker:
            secondary_speaker = r_speaker
        else:
            secondary_speaker = l_speaker

        #print (primary_speaker,l_speaker,r_speaker,speaker_on_focus)

        file_df = pd.read_csv(each_file,sep="\t")
        file_df.columns = [el.replace(primary_speaker, "PRIMA").replace(secondary_speaker, "SECON") for el in file_df.columns]

        if primary_speaker != speaker_on_focus:
            #FILES TO TRAIN!
            train_files.append(file_df)
        else:
            #FILES TO TEST!
            test_files.append(file_df)
            
        
        
    training_data = pd.concat(train_files, ignore_index=True)
    testing_data = pd.concat(test_files, ignore_index=True)
    print ("Speaker on focus:",speaker_on_focus)
    print ("Training data size:",training_data.shape)
    print ("Testing data size:",testing_data.shape)
    print ()
    
    
    
    
    
    
    
    #Train data
    features=['PRIMA:velocity-r', 'PRIMA:velocity-clock', 'PRIMA:velocity-x', 'PRIMA:velocity-y',
              'PRIMA:acceleration-r', 'PRIMA:acceleration-clock', 'PRIMA:acceleration-x',
              'PRIMA:acceleration-y', 'PRIMA:jerk-r', 'PRIMA:jerk-clock', 'PRIMA:jerk-x', 'PRIMA:jerk-y',
              'PRIMA:pitch', 'PRIMA:intensity'
             ]

    feats_learning = training_data[features]
    X = feats_learning.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    scaler = MinMaxScaler((0,1))
    scaler.fit(X)
    X_norm = scaler.transform(X)
    #X_norm = X
    #We're ignoring anyrow that has a NaN value.
    Y = (training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
    #Y = training_data[(feats_learning.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    #Test data
    feats_testing = testing_data[features]

    X_test = feats_testing.dropna() #REMEMBER THIS!!!! We're ignoring any row that has a NaN value.
    X_test_norm = scaler.transform(X_test)
    #X_test_norm = X_test
    Y_test = (testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].notna()*1).values #0: no movement; 1: movement
#    Y_test =  testing_data[(feats_testing.isnull().sum(axis=1)==0)]['PRIMA:HeadMovement'].map(map_movements)
    
    classifiers_to_test = [(DummyClassifier(strategy="most_frequent"),"MF"),
                           (LogisticRegression(solver="liblinear"),"LR"),
                          (LinearSVC(),"LINEARSVC"),
                          (MLPClassifier(hidden_layer_sizes=(30,30,30,30), random_state=442), "MLP")
                          ]
                        #I think that now we have to fine-tune the n_hidden_layers and their sizes (MLP)
    
    
    for clf, clfname in classifiers_to_test:
        print ("Trying classifier ",clfname,speaker_on_focus)
        clf.fit(X_norm, Y)
        y_pred = clf.predict(X_test_norm)
        accuracy_results.loc[clfname,speaker_on_focus] = accuracy_score(Y_test,y_pred)
        #f1score_resultsb.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred)
        f1score_results.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='macro')
        f1score_resultsw.loc[clfname,speaker_on_focus] = f1_score(Y_test,y_pred, average='weighted')
    
        print (confusion_matrix(Y_test,y_pred))
        print (accuracy_score(Y_test,y_pred)) #same as f1-score (micro-averagaed)
        print (f1_score(Y_test,y_pred, average='macro'))
        print (f1_score(Y_test,y_pred, average='weighted'))
        y_pred=None
        print ()

        
accuracy_results

Speaker on focus: F5
Training data size: (181647, 28)
Testing data size: (16413, 28)

Trying classifier  MF F5


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[12154     0]
 [ 4259     0]]
0.7405105708889295
0.42545594567157907
0.6301092504347008

Trying classifier  LR F5
[[11421   733]
 [ 3186  1073]]
0.7612258575519405
0.6036945089926578
0.7238829509017742

Trying classifier  LINEARSVC F5
[[11519   635]
 [ 3285   974]]
0.7611649302382258
0.5932793130752054
0.7189746139288845

Trying classifier  MLP F5
[[10958  1196]
 [ 2792  1467]]
0.7570218729056236
0.6349564386290438
0.7364954338469974

Speaker on focus: M6
Training data size: (181528, 28)
Testing data size: (16532, 28)

Trying classifier  MF M6


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[9316    0]
 [7216    0]]
0.5635131865473022
0.36041473228102755
0.4061969085325493

Trying classifier  LR M6
[[8337  979]
 [4919 2297]]
0.6432373578514397
0.5882801164118258
0.607387766581228

Trying classifier  LINEARSVC M6
[[8418  898]
 [5135 2081]]
0.6350713767239293
0.5722162164179195
0.5930456051890448

Trying classifier  MLP M6
[[7956 1360]
 [3852 3364]]
0.6847326397290104
0.6583752569576407
0.6704289381030841

Speaker on focus: M4
Training data size: (181548, 28)
Testing data size: (16512, 28)

Trying classifier  MF M4


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[[10315     0]
 [ 6152     0]]
0.6264043237991134
0.38514674034799495
0.48251516690223695

Trying classifier  LR F2
[[8757 1558]
 [2635 3517]]
0.7453695269326532
0.7166809225249848
0.7394730332404664

Trying classifier  LINEARSVC F2
[[8961 1354]
 [2828 3324]]
0.7460375296046639
0.7123269449983608
0.7372226632378569

Trying classifier  MLP F2


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[[7604 2711]
 [1819 4333]]
0.7249043541628712
0.7136038100941061
0.7279860066573189



,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.761226,0.643237,0.709666,0.657688,0.746770,0.656137,0.755422,0.758747,0.771266,0.703308,0.753014,0.745370
LINEARSVC,0.761165,0.635071,0.704639,0.651411,0.746160,0.648843,0.751167,0.757545,0.766433,0.700223,0.751515,0.746038
MLP,0.757022,0.684733,0.720688,0.679627,0.758414,0.692378,0.755540,0.749008,0.781798,0.700158,0.760573,0.724904


In [29]:
accuracy_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.740511,0.563513,0.648316,0.588701,0.692986,0.551088,0.591385,0.719731,0.671644,0.506497,0.714565,0.626404
LR,0.761226,0.643237,0.709666,0.657688,0.746770,0.656137,0.755422,0.758747,0.771266,0.703308,0.753014,0.745370
LINEARSVC,0.761165,0.635071,0.704639,0.651411,0.746160,0.648843,0.751167,0.757545,0.766433,0.700223,0.751515,0.746038
MLP,0.757022,0.684733,0.720688,0.679627,0.758414,0.692378,0.755540,0.749008,0.781798,0.700158,0.760573,0.724904


In [30]:
f1score_resultsw

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.630109,0.406197,0.509992,0.436293,0.567316,0.391594,0.439536,0.602434,0.539715,0.340577,0.595607,0.482515
LR,0.723883,0.607388,0.679545,0.601107,0.732226,0.603900,0.746391,0.738121,0.750249,0.701074,0.705285,0.739473
LINEARSVC,0.718975,0.593046,0.668149,0.587914,0.726610,0.590321,0.739132,0.731396,0.741134,0.696555,0.698766,0.737223
MLP,0.736495,0.670429,0.708586,0.646864,0.749277,0.666771,0.753727,0.750025,0.774630,0.700156,0.734271,0.727986


In [31]:
f1score_results

,F5,M6,M4,F3,M1,M5,F4,M3,F1,F6,M2,F2
MF,0.425456,0.360415,0.393320,0.370555,0.409328,0.355291,0.371616,0.418514,0.401786,0.336208,0.416762,0.385147
LR,0.603695,0.588280,0.627744,0.565633,0.671567,0.586572,0.731865,0.654957,0.700095,0.700712,0.598308,0.716681
LINEARSVC,0.593279,0.572216,0.611857,0.550022,0.661058,0.571756,0.722982,0.642035,0.687008,0.696095,0.586824,0.712327
MLP,0.634956,0.658375,0.670394,0.620217,0.696228,0.655203,0.743627,0.691476,0.737719,0.700157,0.649699,0.713604


In [32]:
accuracy_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.634612,0.721821,0.718351,0.730404
std,0.075262,0.046450,0.048769,0.034604
min,0.506497,0.643237,0.635071,0.679627
25%,0.582404,0.691903,0.688020,0.698213
50%,0.637360,0.746070,0.746099,0.736956
75%,0.698381,0.756253,0.753023,0.757370
max,0.740511,0.771266,0.766433,0.781798


In [33]:
f1score_results.T.describe()

,MF,LR,LINEARSVC,MLP
count,12.000000,12.000000,12.000000,12.000000
mean,0.387033,0.645509,0.633955,0.680971
std,0.028472,0.057792,0.060778,0.039306
min,0.336208,0.565633,0.550022,0.620217
25%,0.368020,0.595801,0.583172,0.653827
50%,0.389234,0.641351,0.626946,0.680935
75%,0.411186,0.700250,0.689279,0.703519
max,0.425456,0.731865,0.722982,0.743627
